<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --- 1. Environment Class (with methods restored) ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = total_sim_steps / (365 * days_per_cycle)
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        seasonal_amplitude_mod = 0.225 * np.sin(np.linspace(0, 2 * np.pi * num_years, total_sim_steps)) + 0.725
        self.uv_intensity = np.array([daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(total_sim_steps)])
        self.external_ntp_pool, self.external_aa_pool, self.external_trp_tyr_cys_pool = 0, 0, 0

    # --- METHOD RESTORED ---
    def get_current_uv(self):
        return self.uv_intensity[self.time] if self.time < self.total_sim_steps else 0.0

    # --- METHOD RESTORED ---
    def step(self):
        self.time += 1

# --- 2. RNA_Peptide_Complex Class (with Predation) ---
class RNA_Peptide_Complex:
    def __init__(self, env, strategy='cautious'):
        self.env, self.strategy = env, strategy
        self.rna_pool, self.atp_pool = 100.0, 50.0
        self.mcyw_peptide_pool, self.other_peptide_pool = 10.0, 5.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 100.0, 100.0, 50.0

        self.rna_damage_level, self.protein_synthesis_active = 0.0, True
        self.uv_safety_status, self.premature_stop_signal = "SAFE", 0.0
        self.mcyw_synthesis_bias = 0.0 if strategy == 'generic' else 0.75

        # Parameters tuned for predation dynamics
        self.uv_damage_rate_per_uv = 0.50
        self.repair_rate = 0.4
        self.premature_stop_signal_threshold = 0.20
        self.precarious_codon_rate_per_damage = 1.5
        self.uv_protection_factor = 0.01
        self.damage_tolerance_threshold = 4.0
        self.trp_tyr_cys_for_repair_needed = 0.07
        self.scavenge_efficiency = 0.3

        self.atp_generation_rate_per_mcyw_uv=0.15; self.repair_atp_cost=0.005;
        self.protein_synthesis_rate=0.20; self.protein_synthesis_ntp_cost=0.1; self.protein_synthesis_aa_cost=0.1; self.protein_synthesis_atp_cost=0.02
        self.rna_degradation_rate=0.0008; self.peptide_degradation_rate=0.0015; self.atp_degradation_rate=0.003

    def _apply_uv_damage(self):
        uv = self.env.get_current_uv()
        if uv > 0:
            protection = self.mcyw_peptide_pool * self.uv_protection_factor
            effective_uv = uv * max(0, 1 - protection)
            damage_increase = effective_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else: self.premature_stop_signal = 0

    def _manage_state(self):
        stop_signal = self.premature_stop_signal
        if self.strategy == 'readthrough': stop_signal = 0
        norm_dmg = min(1., self.rna_damage_level / self.damage_tolerance_threshold)
        scarcity = 1. - (self.internal_trp_tyr_cys_stock / 20)
        danger = (norm_dmg * 0.4) + (max(0, min(1, scarcity)) * 0.1) + (stop_signal * 0.5)
        if danger >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active = False; self.uv_safety_status = "ARRESTED"
        elif danger < self.premature_stop_signal_threshold * 0.5:
            self.protein_synthesis_active = True; self.uv_safety_status = "SAFE"
        else: self.uv_safety_status = "CAUTIOUS"
